# Exploring CNNs

This notebook will help me experiment with CNNs to classify origami photos.

In [1]:
# Libraries
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import tensorflow.keras as keras
import tensorboard
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import h5py
import datetime
%load_ext tensorboard
!rm -rf ./logs/

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


KeyboardInterrupt: 

The data are stored locally in a PostgreSQL database. So let's connect and get them.

In [2]:
# Connect to PostgreSQL
user = 'wwatson'
host = 'localhost'
dbname = 'origami'
db = create_engine('postgres://%s%s/%s'%(user, host, dbname))
con = None
con = psycopg2.connect(database=dbname, user=user)
# Execute query
sql_query = "SELECT image_path AS filename, image_class AS class FROM origami_images;"
query_results = pd.read_sql_query(sql_query, con)
# query_results.head()

Let's quickly generate an array of class names; it'll be useful later.

In [3]:
class_names = query_results['class'].unique()

With our dataframe in hand, we'll use an `ImageDataGenerator` from the `keras` API to process and augment the data. The documentation says that this can be relatively slow, but I don't think I have enough data to make that worth worrying about.

In [4]:
# Initialize ImageDataGenerator object, normalizing pixel values as floats in [0, 1]
# TODO: reinsert parameter: validation_split = 0.2
train_datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=40,
                                                               width_shift_range=0.2,
                                                               height_shift_range=0.2,
                                                               rescale=1./255,
                                                               shear_range=0.2,
                                                               zoom_range=0.2,
                                                               horizontal_flip=True,
                                                               fill_mode='nearest')
# For later
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Set batch size, image resolution, and steps per epoch
IMG_SIZE = 64
BATCH_SIZE = 32
image_count = query_results.shape[0]
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

# Generate data from images
train_generator = train_datagen.flow_from_dataframe(query_results,
                                                    x_col = 'filename',
                                                    y_col = 'class',
                                                    target_size = (IMG_SIZE, IMG_SIZE),
                                                    color_mode = 'grayscale',
                                                    batch_size = BATCH_SIZE)

Found 1140 validated image filenames belonging to 6 classes.


/Users/wwatson/opt/anaconda3/envs/origami/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 9 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Now let's construct our model.

In [5]:
cnn = keras.Sequential()
# Layer construction TODO: add L2 regularizers on convolutional layers
cnn.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
cnn.add(keras.layers.MaxPooling2D((2, 2)))
cnn.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(keras.layers.MaxPooling2D((2, 2)))
cnn.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
cnn.add(keras.layers.Flatten())
cnn.add(keras.layers.Dense(32, activation='relu'))
cnn.add(keras.layers.Dense(len(class_names), activation='softmax'))

And let's compile it.

In [6]:
cnn.compile(optimizer='RMSprop',
           loss=keras.losses.CategoricalCrossentropy(from_logits=True),
           metrics=[keras.losses.CategoricalCrossentropy(from_logits=True),
                    'categorical_accuracy'])
cnn.summary()

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                2

And finally fit it.

In [ ]:
history = cnn.fit(train_generator,
                  steps_per_epoch=len(train_generator.filenames) // BATCH_SIZE,
                  epochs=16,
                  callbacks=[tensorboard_callback])

Train for 35 steps
Epoch 1/16


In [4]:
query_results.shape[0]

1149

In [10]:
len(class_names)

6